In [5]:
import sys
sys.path.insert(0, '../')

import src.utils as util
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [6]:
import matplotlib as mpl
import matplotlib.pyplot as plt

In [7]:
import pandas as pd
import pickle
import numpy as np

import plotly.graph_objs as go
from plotly.offline import init_notebook_mode, iplot
init_notebook_mode(connected=True)

In [8]:
import src.post_process as pp
import src.calculate_errors as err
import src.utils as util

Calculate Errors from the models

In [9]:
data = pd.read_pickle('../input/solar_timeseries')

In [10]:
test_actual = data['grid'][int(len(data)*0.9):]

In [11]:
len(test_actual)

6235

Loss Graph

In [14]:
loss = []
trace_names = []
for col in data.columns[0:1]:
    nn_loss = pd.read_pickle(f'../fc_results/training_loss_{str(col)}')
    loss.append(nn_loss)
    trace_names.append(str(col))

In [15]:
util.plot_multiple_loss(loss, trace_names)

Forecast Results

In [55]:
test_actual

date_str
2021-01-13 16:25:00    293507.63
2021-01-13 16:30:00    293417.39
2021-01-13 16:35:00    292774.70
2021-01-13 16:40:00    290386.24
2021-01-13 16:45:00    291626.78
                         ...    
2021-02-14 21:55:00      4329.13
2021-02-14 22:00:00      2464.45
2021-02-14 22:05:00      1448.96
2021-02-14 22:10:00       450.37
2021-02-14 22:15:00        92.45
Name: grid, Length: 6235, dtype: float64

In [49]:
fc_df = pp.combine_hf_fc(data, 3)

In [56]:
fc_df = fc_df.round(2)

In [57]:
fc_df.head()

,grid,tline_1,tline_2,substation1,substation2,substation3,substation4,6010,6014,6011,...,1004972352,143824576,368424646,1428227531,2040495184,1893337300,2117861717,1941427679,1708098535,456209269
date_str,,,,,,,,,,,,,,,,,,,,,
2021-01-13 16:25:00,292470.61,83493.00,211126.91,62903.84,20327.27,204804.89,4292.04,31862.54,30695.38,20262.92,...,4147.29,2477.43,31152.68,1867.63,3651.52,4398.97,7931.46,5315.05,4864.87,5016.53
2021-01-13 16:30:00,293274.87,82491.75,210599.80,62153.73,20107.97,206025.36,4174.54,31119.39,30711.07,20026.88,...,4125.80,2358.13,31194.66,1831.65,3665.89,4260.08,7877.86,5357.55,4800.09,4836.32
2021-01-13 16:35:00,291998.86,82349.54,210097.34,62092.00,20111.80,204960.81,4107.77,30956.36,30740.95,19985.00,...,4038.73,2356.24,31249.32,1833.16,3639.66,4210.64,7848.79,5341.51,4800.98,4888.94
2021-01-13 16:40:00,290478.99,82295.57,208927.57,61927.51,20065.41,203055.79,4024.44,30881.93,30655.61,19912.94,...,4021.51,2344.16,31057.59,1829.36,3592.20,4172.70,7807.51,5309.37,4762.92,4898.84
2021-01-13 16:45:00,289934.50,82544.45,207167.76,61940.31,20342.08,203243.08,4006.86,30731.37,30503.63,20429.08,...,4108.19,2411.87,30851.97,1809.96,3570.80,4061.25,7972.69,5357.97,4719.10,5048.65


In [60]:
mase_vals, mase_ls = err.calculate_errors(data, fc_df, 1, 3)

In [62]:
len(mase_vals)

39

In [126]:
# fig = go.Figure()
# for col in fc_df.keys():
#     box_data = mase_ls[col]
#     fig.add_trace(go.Box(y=box_data, name = col))
# fig.update_xaxes(title = 'time series')
# fig.update_yaxes(title = 'MASE')
# fig.update_layout(showlegend = False)

In [70]:
# fig = go.Figure()
# fig.add_trace(go.Bar(x = fc_df.columns.tolist(), y=mase_vals))
# fig.update_xaxes(title = 'time series')
# fig.update_yaxes(title = 'mean MASE')

Calculate the errors at the top level of the hierarchy

In [73]:
grid_fc = fc_df[['grid']]    

In [89]:
tl_level = fc_df.iloc[:,[1,2]]
substation_level = fc_df.iloc[:,3:7]
pc_level = fc_df.iloc[:,7:13]
house_level = fc_df.iloc[:,13:]

In [94]:
tl_level_df = pd.DataFrame(tl_level.sum(axis=1), columns = ['tl'])
substation_level_df = pd.DataFrame(substation_level.sum(axis=1), columns = ['substation'])
pc_level_df = pd.DataFrame(pc_level.sum(axis=1), columns = ['pc_level'])
house_level_df = pd.DataFrame(house_level.sum(axis=1), columns = ['house'])

In [95]:
grid_fc = pd.concat([grid_fc, tl_level_df, substation_level_df, pc_level_df, house_level_df], axis=1)

In [102]:
mase_ls_grid = []
mase_dic = {}

train = data['grid'][0:int(len(data)*0.7)]
denom = err.calculate_denom(train, 1)

for col in grid_fc.columns:
    forecasts = grid_fc[col].values
    mase, mase_arr = err.test_errors(train, test_actual, forecasts, 3, 1, denom)
    mase_ls_grid.append(mase)
    mase_dic[col] = mase_arr

In [103]:
mase_ls_grid

[0.9609531942355383,
 0.9452244281536608,
 0.9605070821568917,
 1.014553832701932,
 1.0255598367623544]

In [112]:
# mase_dic['grid']

In [108]:
# fig = go.Figure()
# fig.add_trace(go.Bar(x = ['TD', 'Transimission Zone Level (BU)', 'Substation Level (BU)', 
#                           'Postcode Level (BU)', 'House Level (BU)'], y=mase_ls_grid))
# fig.update_xaxes(title = 'Forecast strategy')
# fig.update_yaxes(title = 'mean MASE')

In [119]:
# dic_conv = {'grid': 'TD', 'tl':'Transimission Zone Level (BU)', 'substation': 'Substation Level (BU)', 
#                           'pc_level':'Postcode Level (BU)', 'house':'House Level (BU)'}

# fig = go.Figure()
# for col in mase_dic.keys():
#     box_data = mase_dic[col]
#     fig.add_trace(go.Box(y=box_data, name = dic_conv[col]))
# fig.update_xaxes(title = 'Forecast strategy')
# fig.update_yaxes(title = 'MASE')

Plot the forecasts

In [23]:
def plot_fc(data, fc):
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=data.index, y=data.values, name ='target generation'))
    fig.add_trace(go.Scatter(x=fc.index, y=fc.values, name ='forecasts'))
    fig.show()

In [39]:
# plot_fc(test_actual, fc['grid'])

In [22]:
train = data['grid'][int(len(data)*0.7):]

In [34]:
dm = err.calculate_denom(train, 1)

In [35]:
dm

4308.036596647299

In [40]:
mase, error = err.test_errors(train, test_actual, fc['grid'].values, 3, 1, dm)

In [46]:
len(train)

31796

In [41]:
mase

1.654953800219155

In [45]:
fig = go.Figure()
fig.add_trace(go.Box(y=error, name = 'fc - grid level'))

New Data Errors

In [8]:
df = pd.read_pickle('../input/ts_1h')

In [9]:
df = df.resample('1D').mean()

In [10]:
val_start = pd.Timestamp("2021-01-31")
val_end = pd.Timestamp("2021-02-14")

In [11]:
df = df.reset_index()

In [12]:
df.head()

,date_str,grid,tline_1,tline_2,substation1,substation2,substation3,substation4,6010,6014,...,1004972352,143824576,368424646,1428227531,2040495184,1893337300,2117861717,1941427679,1708098535,456209269
0,2020-02-13,157324.643571,39616.774286,117707.867857,29664.033571,9952.740714,115604.292143,2103.575000,15521.148571,14142.885714,...,1983.175000,1106.558571,31025.717857,877.119286,1862.292143,2103.575000,3903.341429,2305.494286,2440.205000,2325.777143
1,2020-02-14,165094.829286,45400.262857,119694.568571,33777.835000,11622.427857,117467.982143,2226.584286,17530.031429,16247.805000,...,2072.195714,1323.740000,31468.928571,849.185714,1903.065714,2226.584286,4573.934286,2663.396429,2723.791429,2654.098571
2,2020-02-15,110615.075714,17172.801429,93442.275000,13700.127857,3472.671429,91657.050000,1785.224286,7123.702857,6576.425714,...,1876.244286,513.909286,15771.560714,817.257143,1847.200000,1785.224286,1733.961429,1087.384286,1136.372857,1006.577143
3,2020-02-16,133407.398571,45007.996429,88399.402857,33508.278571,11499.717143,86940.897857,1458.507143,17281.965714,16226.312143,...,1343.883571,1309.204286,19468.955714,528.428571,1158.050000,1458.507143,4458.514286,2816.281429,2590.491429,2703.689286
4,2020-02-17,177933.992857,45992.875000,131941.119286,34224.037143,11768.836429,129483.499286,2457.620714,18011.540000,16212.498571,...,2335.563571,1379.542857,34520.867143,956.817857,2063.695000,2457.620714,4670.427143,2725.972143,2782.651429,2718.415714


In [13]:
test = df.loc[df['date_str']>=val_start]

In [14]:
train = df.loc[df['date_str']<val_start]['grid']

In [15]:
mean = 110852.73482653

In [7]:
cnn_1 = pd.read_csv('../fc_results/cnn_results.csv', index_col = [0])+mean

In [22]:
cnn_1

,target,cnn-1,dilation-2,dilation-3,dilation-5
0,160728.371429,151911.039514,136951.519983,142029.555139,154962.691858
1,166689.189286,135160.857873,136029.236780,138843.844202,154204.250452
2,147175.335714,136548.098108,135684.809045,135422.107873,162605.117639
3,162933.855000,174222.742639,149049.055139,144454.820764,182452.391077
4,179447.513571,169430.016077,154219.402795,156040.602014,189911.555139
5,170105.036429,168720.285608,156401.168420,168850.988733,215178.883264
6,52420.040714,151121.699670,143084.123498,133185.557092,171406.730920
7,30890.923571,138004.666467,132703.545373,137849.803186,168895.613733
8,124078.987857,136001.557092,137699.687952,121560.724084,164855.652795
9,135138.137857,138406.129358,144989.887170,133881.541467,166990.316858


In [32]:
cnn_exg = pd.read_csv('../fc_results/cnnexg.csv', index_col = [0])+mean

In [33]:
cnn_exg

,cnn-1,dilation-2,dilation-3,dilation-5
0,164571.184045,178075.219202,173818.891077,178941.883264
1,142148.986780,185067.148889,176836.750452,184057.328577
2,135429.506311,159192.109827,154726.879358,160898.344202


In [39]:
cnn_2 = pd.read_csv('../fc_results/wavenet_results.csv', index_col = [0])+mean

In [40]:
cnn_exg2 = pd.read_csv('../fc_results/wavenetexg.csv', index_col = [0])+mean

In [16]:
denom = err.calculate_denom(train, 3)

In [17]:
denom

25143.60411120129

In [29]:
mase_ls_grid1 = []
mase_dic ={}

for col in cnn_1.columns[1:]:
    forecasts = cnn_1[col][0:3].values
    test_actual = cnn_1['target'][0:3].values
    mase, mase_arr = err.test_errors(train, test_actual, forecasts, 3, 1, denom)
    mase_ls_grid1.append(mase)
    mase_dic[col] = mase_arr

In [30]:
mase_dic

{'cnn-1': [0.6757570196635202],
 'dilation-2': [0.8740106142471479],
 'dilation-3': [0.7728590931352518],
 'dilation-5': [0.4465071936487011]}

In [31]:
mase_ls_grid1

[0.6757570196635202,
 0.8740106142471479,
 0.7728590931352518,
 0.4465071936487011]

In [24]:
cnn_1['target'][0:3].values

array([160728.37142857, 166689.18928571, 147175.33571429])

In [34]:
mase_ls_gridexg = []

for col in cnn_exg.columns:
    forecasts = cnn_exg[col].values
    test_actual = cnn_1['target'][0:3].values
    mase, mase_arr = err.test_errors(train, test_actual, forecasts, 3, 1, denom)
    mase_ls_gridexg.append(mase)

In [35]:
mase_ls_gridexg

[0.5319953913352041,
 0.6329188300051228,
 0.40818365207441887,
 0.6536408410327524]

In [65]:
# fig = go.Figure()
# for col in mase_dic.keys():
#     box_data = mase_dic[col]
#     fig.add_trace(go.Box(y=box_data, name = col))
# fig.update_xaxes(title = 'TCN')
# fig.update_yaxes(title = 'MASE')

In [41]:
mase_ls_grid2 = []
# mase_dic ={}

for col in cnn_2.columns[1:]:
    forecasts = cnn_2[col][0:3].values
    test_actual = cnn_2['target'][0:3].values
    mase, mase_arr = err.test_errors(train, test_actual, forecasts, 3, 1, denom)
    mase_ls_grid2.append(mase)
    mase_dic["wv-" + col] = mase_arr

In [42]:
mase_ls_grid2

[0.29788330951341574,
 0.4123980699808946,
 0.32417530480214324,
 0.38846378473023413]

In [46]:
mase_ls_grid2exg

[0.48976351389843575,
 0.9998236785245584,
 0.27633909427862857,
 0.8246981699117457]

In [45]:
mase_ls_grid2exg = []
# mase_dic ={}

for col in cnn_exg2.columns:
    forecasts = cnn_exg2[col].values
    test_actual = cnn_2['target'][0:3].values
    mase, mase_arr = err.test_errors(train, test_actual, forecasts, 3, 1, denom)
    mase_ls_grid2exg.append(mase)
#     mase_ls_grid2exg["wv-" + col] = mase_arr

In [80]:
arima = pd.read_csv('../fc_results/arima.csv', index_col = [0])

In [82]:
mase_ls_grid3 = []
# mase_dic ={}

for col in arima.columns:
    forecasts = arima[col].values
    test_actual = cnn_2['target'].values
    mase, mase_arr = err.test_errors(train, test_actual, forecasts, 3, 1, denom)
    mase_ls_grid3.append(mase)
    mase_dic[col] = mase_arr

In [83]:
mase_ls_grid3

[1.2585419615505855]

In [90]:
rnn = pd.read_pickle('../fc_results/forecast_rnn')

In [92]:
fc_ls = []
for batch in rnn:
    batch_data = batch[::3]
    for h in batch_data:
        fc_ls.extend(h.tolist())

In [98]:
rnn_df = pd.DataFrame(fc_ls[-15:], columns = ['rnn'])

In [99]:
train_df = train[0:-50]

In [102]:
rnn_df = (rnn_df * train_df.std()) + train_df.mean()

In [104]:
mase_ls_grid4 = []
# mase_dic ={}

for col in rnn_df.columns:
    forecasts = rnn_df[col].values
    test_actual = cnn_2['target'].values
    mase, mase_arr = err.test_errors(train, test_actual, forecasts, 3, 1, denom)
    mase_ls_grid4.append(mase)
    mase_dic[col] = mase_arr

In [106]:
mase_ls_grid4

[1.1571992315721709]

In [114]:
for key in mase_dic:
    print(key)
    print(np.median(mase_dic[key]))
    print(mase_dic[key])

cnn-1
0.48122999224131485
[0.6757570196635202, 0.3008205168779632, 2.886591889316485, 0.48122999224131485, 0.22690731682385337]
dilation-2
0.7002016418942705
[0.8740106142471479, 0.7002016418942705, 2.732270783045378, 0.5418942957152165, 0.5031088427547701]
dilation-3
0.5719147408669689
[0.7728590931352518, 0.5719147408669689, 2.5220815457186183, 0.4429888753227469, 0.17558431210293843]
dilation-5
0.9950366721180716
[0.4465071936487011, 0.9950366721180716, 3.947565140735351, 1.5388451891276411, 0.875240954965188]
wv-wavenet-1
0.6197897907905824
[0.29788330951341574, 0.6197897907905824, 3.5169342242858517, 0.5676275732870565, 0.6660521944701009]
wv-dilation-2
0.5465826491323335
[0.4123980699808946, 0.704489149164107, 3.4754557128433046, 0.4264044073759245, 0.5465826491323335]
wv-dilation-2/k4
0.5628590155596797
[0.32417530480214324, 0.3685882789582973, 3.8220361478302625, 0.6998510718084705, 0.5628590155596797]
wv-dilation-3
0.6975187197071812
[0.38846378473023413, 0.6975187197071812, 3

In [105]:
labels = ['tcn', 'tcn-d^2', 'tcn-d^3', 'tcn-d^5', 'wv', 'wv-d^2', 'wv-d^2/k4', 'wv-d^3', 'auto-arima', 'rnn']

fig = go.Figure()
i=0
for col in mase_dic.keys():
    box_data = mase_dic[col]
    fig.add_trace(go.Box(y=box_data, name = labels[i], boxmean=True))
    i = i+1
fig.update_xaxes(title = 'Convolution Networks')
fig.update_yaxes(title = 'MASE')
fig.update_layout(showlegend = False)